# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Manipulating').getOrCreate()
spark

## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [4]:
tweets = spark.read.csv('Datasets/Rep_vs_Dem_tweets.csv',inferSchema=True, header=True)
print(tweets.printSchema())
tweets.limit(5).toPandas()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)

None


,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...


In [3]:
tweets.printSchema()

== Physical Plan ==
FileScan csv [Party#16,Handle#17,Tweet#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/antonhaugen/Desktop/FlatironProjects/pysparkpractice/Section_3/Data..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Party:string,Handle:string,Tweet:string>




## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [8]:
tweets.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...


**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [10]:
tweets.show(10, False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [11]:
tweets.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [57]:
from pyspark.sql.functions import regexp_replace, regexp_extract

In [35]:
tweets.show(10, False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

In [56]:
tweets.select('Tweet', regexp_extract('Tweet', '(@LatinoLeader)',1)).where(regexp_extract('Tweet', '(@LatinoLeader)',1)=='@LatinoLeader').limit(100).toPandas()

,Tweet,"regexp_extract(Tweet, (@LatinoLeader), 1)"
0,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,@LatinoLeader


## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [59]:
tweets.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...


In [77]:
tweets = tweets.withColumn('Party', regexp_replace(tweets.Party, '^[^(Democrat)(Republican)].*$', 'Other'))

In [78]:
tweets.select('Party').limit(10).toPandas()

,Party
0,Democrat
1,Democrat
2,Democrat
3,Other
4,Democrat
5,Democrat
6,Democrat
7,Democrat
8,Democrat
9,Democrat


## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [80]:
tweets.show(5,False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+
|Party   |Handle       |Tweet                                                                                                                                       |Party_test|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |Democrat  |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|Democrat  |
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billio

In [82]:
tweets = tweets.withColumn('Tweet', regexp_replace(tweets.Tweet, 'https://.*', ''))
tweets.show(5, False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+
|Party   |Handle       |Tweet                                                                                                                                       |Party_test|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |Democrat  |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|Democrat  |
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billio

## 4. Remove any leading or trailing white space in the tweet column

In [84]:
from pyspark.sql.functions import *

In [85]:
tweets = tweets.withColumn('Tweet', trim(tweets.Tweet))

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [86]:
tweets = tweets.withColumnRenamed('Party','Dem_Rep')
tweets.show(5, False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+
|Dem_Rep |Handle       |Tweet                                                                                                                                       |Party_test|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |Democrat  |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|Democrat  |
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billio

In [96]:
tweets = tweets.drop('Party_test')

## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [97]:
tweets= tweets.withColumn('Concat', concat_ws(' ', tweets['Dem_Rep'], tweets['Handle']))

In [98]:
tweets.show(5)

+--------+-------------+--------------------+--------------------+
| Dem_Rep|       Handle|               Tweet|              Concat|
+--------+-------------+--------------------+--------------------+
|Democrat|RepDarrenSoto|Today, Senate Dem...|Democrat RepDarre...|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|Democrat RepDarre...|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@...|Democrat RepDarre...|
|   Other|         null|                null|               Other|
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|Democrat RepDarre...|
+--------+-------------+--------------------+--------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [160]:
tweets= tweets.withColumn('Hashtags', regexp_extract('Tweet', '(#\w*)',1))

In [175]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import re


# Define the function you want to return
def extract(s):
    if s:
        all_matches = re.findall(r'(#\w*)', s)
        return all_matches
    else:
        return None

# Create the UDF, note that you need to declare the return schema matching the returned type
extract_udf = udf(lambda z : extract(z), ArrayType(StringType()))

# Apply it
tweets = tweets.withColumn('Hashtags', extract_udf(tweets.Tweet))

In [176]:

tweets.show(10, False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------------------+----------------------------------+
|Dem_Rep |Handle       |Tweet                                                                                                                                       |Concat                |Hashtags                          |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------------------+----------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |Democrat RepDarrenSoto|[#SaveTheInternet, #NetNeutrality]|
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [209]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [214]:
df.select('dob', to_date(concat_ws('-0',split('dob', '-')))).show(5)

+---------+-----------------------------------------------+
|      dob|to_date(concat_ws('-0', split(`dob`, '-', -1)))|
+---------+-----------------------------------------------+
| 1987-4-8|                                     1987-04-08|
| 1986-4-8|                                     1986-04-08|
|1986-7-10|                                     1986-07-10|
| 1988-5-2|                                     1988-05-02|
|1987-5-11|                                     1987-05-11|
+---------+-----------------------------------------------+
only showing top 5 rows



In [215]:
df = df.withColumn('dob',  to_date(concat_ws('-0',split('dob', '-'))))\
        .withColumn('visit1', to_date(concat_ws('-0',split('visit1', '-'))))\
        .withColumn('visit2', to_date(concat_ws('-0',split('visit2', '-'))))\
        .withColumn('visit3', to_date(concat_ws('-0',split('visit3', '-'))))
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)



In [216]:
df.limit(5).toPandas()

,first_name,last_name,dob,visit1,visit2,visit3
0,Mohammed,Alfasy,1987-04-08,2016-01-07,2017-02-03,2018-03-02
1,Marcy,Wellmaker,1986-04-08,2015-01-07,2017-01-03,2018-01-02
2,Ginny,Ginger,1986-07-10,2014-08-07,2015-02-03,2016-03-02
3,Vijay,Doberson,1988-05-02,2016-01-07,2018-02-03,2018-03-02
4,Orhan,Gelicek,1987-05-11,2016-05-07,2017-01-03,2018-09-02


In [201]:
df = df.withColumn('dob', to_date('dob', 'yyyy-mm-dd'))

In [202]:
df.show(5)

Py4JJavaError: An error occurred while calling o995.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 119.0 failed 1 times, most recent failure: Lost task 0.0 in stage 119.0 (TID 168, 192.168.1.21, executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '1987-4-8' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$ano

## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

## 8. Can you calculate the age of each patient?

## 9. Can you extract the month from the first visit column and call it "Month"?

## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [3]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

### That's it! Great Job!